In [1]:
import torch
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
#from torch import nn

In [3]:
# Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)

inp = torch.randn(1, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))

In [72]:
"""   
class ConvolutionTransposed(object):
    def __init__(self, channels_input, channels_output, kernel_size, stride):
        self.weight = torch.empty(channels_output, channels_input, kernel_size, kernel_size).normal_()
        
        self.channel_input = channels_input
        #print('input channels', self.channel_input)
        self.kernel_size = kernel_size
        self.stride = stride
        
        print('weight',self.weight.shape)
        
        
    def forward(self, imgs):
        #print('forward')
        _,_,H,W = imgs.shape
        H_out = (H - 1)*self.stride + self.kernel_size 
        W_out = (W - 1)*self.stride + self.kernel_size 
        
        #print('Hout Wout', H_out, W_out)
        
        self.x = imgs.permute(1, 2, 3, 0).reshape(self.channel_input, -1)
        #print('x',self.x.shape)
        self.y = (self.weight.reshape(self.channel_input, -1)).t().matmul(self.x)
        #print('y',self.y.shape)
        self.y = self.y.reshape(self.y.shape[0], -1, imgs.shape[0])
        #print('y2',self.y.shape)
        self.y = self.y.permute(2, 0, 1)
        
        #print(self.y.shape)
        self.y = fold( self.y, (H_out, W_out), kernel_size=(self.kernel_size,self.kernel_size), stride=self.stride)
        
        return self.y
"""

"   \nclass ConvolutionTransposed(object):\n    def __init__(self, channels_input, channels_output, kernel_size, stride):\n        self.weight = torch.empty(channels_output, channels_input, kernel_size, kernel_size).normal_()\n        \n        self.channel_input = channels_input\n        #print('input channels', self.channel_input)\n        self.kernel_size = kernel_size\n        self.stride = stride\n        \n        print('weight',self.weight.shape)\n        \n        \n    def forward(self, imgs):\n        #print('forward')\n        _,_,H,W = imgs.shape\n        H_out = (H - 1)*self.stride + self.kernel_size \n        W_out = (W - 1)*self.stride + self.kernel_size \n        \n        #print('Hout Wout', H_out, W_out)\n        \n        self.x = imgs.permute(1, 2, 3, 0).reshape(self.channel_input, -1)\n        #print('x',self.x.shape)\n        self.y = (self.weight.reshape(self.channel_input, -1)).t().matmul(self.x)\n        #print('y',self.y.shape)\n        self.y = self.y.reshape

In [40]:
class ConvolutionTransposed(object):
    def __init__(self,channels_input, channels_output, kernel_size, stride):
        self.device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")
        
        self.weight = torch.empty(channels_output, channels_input, kernel_size, kernel_size).normal_()
        self.grad = torch.empty(channels_output, channels_input, kernel_size, kernel_size)
        
        self.bias = 0
        self.grad_bias = 0
        
        self.kernel_size = kernel_size
        self.stride = stride
        self.channels_output = channels_output
        self.channels_input = channels_input
        
    def forward(self,x):
        
        B,I,SI,SI = x.shape
        SO = (SI -1)*self.stride + self.kernel_size
        self.H = SI
        self.W = SI
        self.x = x
        self.x_reshape = self.x.reshape(B,I,-1) # [B,I,SI,SI]
        self.weight_reshape = self.weight.permute(1,0,2,3).reshape(I,-1) # [I,(OxKxK)]
        
        self.y_reshape = self.weight_reshape.T @ self.x_reshape # [B, OxKxK, SIxSI]
        print('y', self.y_reshape.shape)
        
        self.y = fold(self.y_reshape, kernel_size =(self.kernel_size,self.kernel_size), stride = self.stride, output_size=(SO,SO))
        

        
        
        return self.y
    
    def backward(self,gradwrtoutput):
        
        dL_dS = gradwrtoutput # [B, O, SO, SO]
        print('dL_dS', dL_dS.shape)
        dS_dX = self.weight # [O,I,K,K]
        
        #backward Input
        dL_dS_unfold = unfold(dL_dS, kernel_size =(self.kernel_size,self.kernel_size), stride = self.stride)#[B, OxKxK, SIxSI]
        print('unfold', dL_dS_unfold.shape )
       
        dS_dX_reshape = dS_dX.reshape(self.channels_input,-1) # [I,OxKxK]
        print('dS_dX_reshape', dS_dX_reshape.shape)
        
        dL_dX_reshape = dS_dX_reshape @ dL_dS_unfold # [B,I,SIxSI]
        dL_dX =  dL_dX_reshape.view(1,self.channels_input,self.H, self.W)
        
        #backward weight
        dL_dS_unfold2 = dL_dS_unfold.permute(1,0,2).reshape(self.channels_input*self.kernel_size*self.kernel_size,-1) #[OxKxK, BxSIxSI]
        print('unfold', dL_dS_unfold2.shape )
        dS_dW_reshape = self.x.reshape(self.channels_input, -1 ) #[I, BxSIxSI]
        
        dL_dW_reshape =  dS_dW_reshape @ dL_dS_unfold2.T # [I, OxKxK]
        
        dL_dW = dL_dW_reshape.reshape(self.channels_input, self.channels_output, self.kernel_size, self.kernel_size )
        
        self.grad = dL_dW
        print('self.grad',self.grad.shape)
        
        return dL_dX
    
    def param(self):
        return self.weight, self.grad, self.bias, self.grad_bias 
        


In [46]:
img = torch.randn(1, 3, 14,14)
img_error = torch.randn(1, 3, 32, 32)

convT1 = ConvolutionTransposed(3,3,6,2)

out = convT1.forward(img)

print(out.shape)



y torch.Size([1, 108, 196])
torch.Size([1, 3, 32, 32])


In [47]:
y = convT1.backward(out)
print(y.shape)

dL_dS torch.Size([1, 3, 32, 32])
unfold torch.Size([1, 108, 196])
dS_dX_reshape torch.Size([3, 108])
unfold torch.Size([108, 196])
self.grad torch.Size([3, 3, 6, 6])
torch.Size([1, 3, 14, 14])
